In [1]:
# screening test resolve bioscience

#Author: Marlene Ganslmeier

import random #technically a library but I feel it's too much to ask to write your own randomness algorithm
#it also comes with the standard installation so I think it count's as acceptable 
import csv #same applies

#function to create a random string comprised of the 4 bases with length n
def create_seq(n):
    bases = ["A", "T", "G", "C"]
    seq = []
    seq = [random.choice(bases) for x in range(n)]
    
    return seq

class HashMap:

    def __init__(self, size):
        self.array_size = size
        self.array = [None for i in range(size)]
    
    def indexing(self, key):
        hash_code = key.encode()
        hash_code = sum(hash_code)
        index = hash_code % self.array_size
        return index

    def assign(self, key):
        index = self.indexing(key)

        if self.array[index] == None:
           self.array[index] = [key]
        else:
            self.array[index].append(key)
            return True
    
    def retrieve(self, key):
        index = self.indexing(key)
        list_at_index = self.array[index]

        if list_at_index == None:
            return None
        else:
            for item in list_at_index:
                if item == key:
                    return item
                else:
                    return None

In [5]:

first_slot = HashMap(96)
seq = "".join(create_seq(6))
print(seq[0:3])
first_slot.assign(seq[0:3])



TGT
